In [1]:
import pandas as pd
import random

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
INPUT_FILE = '../toxicity_ratings - twitter.csv'

In [4]:
df = pd.read_csv(INPUT_FILE)

/Users/shagunjhaver/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,7,8,9,10,14,15,16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df = df.dropna()

In [6]:
def get_avg_toxicity(row):
    result = (row['toxic_score'] + row['toxic_score.1'] + row['toxic_score.2'] + row['toxic_score.3'] + row['toxic_score.4'])/20.0
    return result

In [7]:
df['avg_toxic_score'] = df.apply (lambda row: get_avg_toxicity(row), axis=1)

In [8]:
df = df[['comment', 'perspective_score', 'avg_toxic_score', 'comment_ID']]

In [9]:
def get_bucket(row, field):
    if (row[field] < 0.2):
        return 0.0
    elif (row[field] < 0.4):
        return 0.2
    elif (row[field] < 0.6):
        return 0.4
    elif (row[field] < 0.8):
        return 0.6
    else:
        return 0.8

In [10]:
df['perspective_window'] = df.apply(lambda row: get_bucket(row, 'perspective_score'), axis=1)

In [11]:
df['avg_toxic_window'] = df.apply(lambda row: get_bucket(row, 'avg_toxic_score'), axis=1)


In [12]:
f = df[df['perspective_window'] != df['avg_toxic_window']]

In [13]:
len(f)

58321

In [14]:
myrange = [0.0, 0.2, 0.4, 0.6, 0.8]
sampled_dfs = []
for ow in myrange:
    g = f[f['perspective_window'] == ow]
    for iw in myrange:
        if (ow != iw):
            h = g[g['avg_toxic_window'] == iw]
            sampled_dfs.append(h.sample(1))

In [15]:
usernameDf = pd.read_csv('../usernames.csv')
usernames = set(list(usernameDf['username']))

In [16]:
final_df = pd.concat(sampled_dfs)

In [17]:
final_df['author'] = df.apply(lambda row: random.sample(usernames, 1)[0], axis=1)
final_df

,comment,perspective_score,avg_toxic_score,comment_ID,perspective_window,avg_toxic_window,author
44589,Best #ALDUBxADNBeatingTheOdds,0.159522,0.30,2347_23,0.0,0.2,high_and_mighty
40881,"Nae Nae came to Jewels ratchet salon tonight. We do lips, brows and facials",0.073388,0.40,3193_21,0.0,0.4,orderly_reviewer
50475,Cute that she knelt thinking it actually meant something 😂😂,0.137157,0.60,5107_15,0.0,0.6,doting_lineage
69223,Both of them confused cause I treat em like the same hoe,0.166179,0.80,4391_23,0.0,0.8,GiftedInspiration
26531,Oh wow!! I literally just stumbled on your art today!,0.249317,0.10,2770_6,0.2,0.0,MustySeating
3324,I don't think we have to bring those people into this because they've made themselves cleared about their religion,0.379370,0.40,1636_5,0.2,0.4,rebelsolitude666
15010,"I mean, I guess you'd be an expert in ""both sides are the same"" considering you condone pedophilia and sex trafficking of minors just as much as the republicans do, right? Right?",0.366563,0.60,1058_18,0.2,0.6,VirtualStatement
7230,"I love spending my day to spoil my loyal sugar babies... Dm, like or RT if you're ready to be pampered... I will clear up your debt for you if you have a credit card... A weekly payment of $500 right away if i like you. #sugarBabies #sugardaddylegit #CollegeStudent #collegelife",0.224907,0.80,1417_17,0.2,0.8,ValiantlyThorough
69472,HECK YE! We can play when my school break comes,0.526257,0.10,5201_19,0.4,0.0,TheseOrangutan
13550,"Hey, I had TWO wills, BOTH forged and even as a named heir, I never got a copy of it. I gotta post them, as it is the wildest thing and the court-appointed admins and probate court judges let greedy family members get away with murder. Hard to fight w/o legal help; expensive.",0.441413,0.25,1313_9,0.4,0.2,PainfulSquid


In [18]:
final_df.to_csv('main_comments.csv', columns = ['comment', 'author', 'perspective_score', 'avg_toxic_score', 'comment_ID'], index=False)